In [1]:
import math
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch.nn.init as init
# from torch.nn.init import xavier_normal_, xavier_uniform_
import argparse
import numpy as np
import time
import sys
from os.path import abspath
import random
import pandas as pd
from util.sampler import  next_batch_pairwise
from util.conf import OptionConf
import torch
import torch.nn as nn 
import torch.nn.functional as F
from scipy.sparse import coo_matrix
from util.loss_torch import bpr_loss, l2_reg_loss, EmbLoss, contrastLoss, InfoNCE
from util.init import *
from base.torch_interface import TorchGraphInterface
import os
import numpy as np 
import time 
from torch.optim.lr_scheduler import ReduceLROnPlateau
from itertools import product
import collections  
from collections import defaultdict
import scipy.sparse as sp
from itertools import product

from random import shuffle,randint,choice,sample
import torch.nn.init as init 
import csv 

from util.conf import ModelConf
from base.recommender import Recommender
from util.algorithm import find_k_largest
from time import strftime, localtime
from data.loader import FileIO
from util.evaluation import ranking_evaluation

from data.ui_graph import Interaction
from data.augmentor import GraphAugmentor
from util.evaluation import early_stopping


/home/jun/anaconda3/envs/hungvv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda', index=0)

## Base Recommender

In [4]:
class GraphRecommender(Recommender):
    def __init__(self, conf, data, data_kg,knowledge_set, **kwargs):
        super(GraphRecommender, self).__init__(conf, data, data_kg, knowledge_set,**kwargs)
        self.data = data
        self.data_kg = data_kg 
        self.bestPerformance = []
        top = self.ranking['-topN'].split(',')
        self.topN = [int(num) for num in top]
        self.max_N = max(self.topN)
        
        self.dataset = kwargs['dataset']
        
        # self.output = f"./results/{self.model_name}/{self.dataset}/@{self.model_name}-inp_emb:{kwargs['input_dim']}-hyper_emb:{kwargs['hyper_dim']}-bs:{self.batch_size}-lr:{kwargs['lr']}-lrd:{kwargs['lr_decay']}-reg:{kwargs['reg']}-leaky:{kwargs['p']}-dropout:{kwargs['drop_rate']}-n_layers:{kwargs['n_layers']}-n_heads:{kwargs['n_heads']}-n_self_att:{kwargs['n_self_att']}/"
        self.output = f"./results/HGNN_KG_SSL/{self.dataset}/@{self.model_name}-inp_emb:{kwargs['input_dim']}-hyper_emb:{kwargs['hyper_dim']}-bs:{self.batch_size}-lr:{kwargs['lr']}-lrd:{kwargs['lr_decay']}-reg:{kwargs['reg']}-leaky:{kwargs['p']}-dropout:{kwargs['drop_rate']}-n_layers:{kwargs['n_layers']}-cl_rate:{kwargs['cl_rate']}-aug_type:{kwargs['aug_type']}-temp:{kwargs['temp']}/"
        if not os.path.exists(self.output):
            os.makedirs(self.output)

    def print_model_info(self):
        super(GraphRecommender, self).print_model_info()
        # # print dataset statistics
        print('Training Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.training_size()))
        print('Test Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.test_size()))
        print('=' * 80)

    def build(self):
        pass

    def train(self):
        pass

    def predict(self, u):
        pass

    def test(self, user_emb, item_emb):
        def process_bar(num, total):
            rate = float(num) / total
            ratenum = int(50 * rate)
            r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
            sys.stdout.write(r)
            sys.stdout.flush()

        # predict
        rec_list = {}
        user_count = len(self.data.test_set)
        for i, user in enumerate(self.data.test_set):
            # s_find_candidates = time.time()
            
            # candidates = predict(user)
            user_id  = self.data.get_user_id(user)
            score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
            candidates = score.cpu().numpy()
            
            # e_find_candidates = time.time()
            # print("Calculate candidates time: %f s" % (e_find_candidates - s_find_candidates))
            # predictedItems = denormalize(predictedItems, self.data.rScale[-1], self.data.rScale[0])
            rated_list, li = self.data.user_rated(user)
            for item in rated_list:
                candidates[self.data.item[item]] = -10e8
            
            # s_find_k_largest = time.time()
            ids, scores = find_k_largest(self.max_N, candidates)
            # e_find_k_largest = time.time()
            # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
            item_names = [self.data.id2item[iid] for iid in ids]
            rec_list[user] = list(zip(item_names, scores))
            if i % 1000 == 0:
                process_bar(i, user_count)
        process_bar(user_count, user_count)
        print('')
        return rec_list

    def evaluate(self, rec_list):
        self.recOutput.append('userId: recommendations in (itemId, ranking score) pairs, * means the item is hit.\n')
        for user in self.data.test_set:
            line = str(user) + ':'
            for item in rec_list[user]:
                line += ' (' + str(item[0]) + ',' + str(item[1]) + ')'
                if item[0] in self.data.test_set[user]:
                    line += '*'
            line += '\n'
            self.recOutput.append(line)
        current_time = strftime("%Y-%m-%d %H-%M-%S", localtime(time.time()))
        # output prediction result
        out_dir = self.output
        file_name = self.config['model.name'] + '@' + current_time + '-top-' + str(self.max_N) + 'items' + '.txt'
        FileIO.write_file(out_dir, file_name, self.recOutput)
        print('The result has been output to ', abspath(out_dir), '.')
        file_name = self.config['model.name'] + '@' + current_time + '-performance' + '.txt'
        self.result = ranking_evaluation(self.data.test_set, rec_list, self.topN)
        self.model_log.add('###Evaluation Results###')
        self.model_log.add(self.result)
        FileIO.write_file(out_dir, file_name, self.result)
        print('The result of %s:\n%s' % (self.model_name, ''.join(self.result)))

    def fast_evaluation(self, epoch, model_cf, model_kg, user_embed, item_embed, kwargs=None):
        print('Evaluating the model...')
        s_test = time.time()
        rec_list = self.test(user_embed, item_embed)
        e_test = time.time() 
        print("Test time: %f s" % (e_test - s_test))
        
        s_measure = time.time()
        measure = ranking_evaluation(self.data.test_set, rec_list, [self.max_N])
        e_measure = time.time()
        print("Measure time: %f s" % (e_measure - s_measure))
        
        if len(self.bestPerformance) > 0:
            count = 0
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            for k in self.bestPerformance[1]:
                if self.bestPerformance[1][k] > performance[k]:
                    count += 1
                else:
                    count -= 1
            if count < 0:
                self.bestPerformance[1] = performance
                self.bestPerformance[0] = epoch + 1
                # try:
                #     self.save(kwargs)
                # except:
                self.save(model_cf, model_kg, user_embed, item_embed)
        else:
            self.bestPerformance.append(epoch + 1)
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            self.bestPerformance.append(performance)
            # try:
            #     self.save(kwargs)
            # except:
            self.save(model_cf, model_kg, user_embed, item_embed)
        print('-' * 120)
        print('Real-Time Ranking Performance ' + ' (Top-' + str(self.max_N) + ' Item Recommendation)')
        measure = [m.strip() for m in measure[1:]]
        print('*Current Performance*')
        print('Epoch:', str(epoch + 1) + ',', '  |  '.join(measure))
        bp = ''
        # for k in self.bestPerformance[1]:
        #     bp+=k+':'+str(self.bestPerformance[1][k])+' | '
        bp += 'Hit Ratio' + ':' + str(self.bestPerformance[1]['Hit Ratio']) + '  |  '
        bp += 'Precision' + ':' + str(self.bestPerformance[1]['Precision']) + '  |  '
        bp += 'Recall' + ':' + str(self.bestPerformance[1]['Recall']) + '  |  '
        # bp += 'F1' + ':' + str(self.bestPerformance[1]['F1']) + ' | '
        bp += 'NDCG' + ':' + str(self.bestPerformance[1]['NDCG'])
        print('*Best Performance* ')
        print('Epoch:fast_evaluation', str(self.bestPerformance[0]) + ',', bp)
        print('-' * 120)
        return measure
    
    def save(self, model_cf, model_kg, best_user_emb, best_item_emb):
        self.best_user_emb, self.best_item_emb = best_user_emb, best_item_emb
        self.save_model(model_cf, mode='cf')
        self.save_model(model_kg, mode='kg')
    
    def save_model(self, model, mode='cf'):
        # save model 
        current_time = strftime("%Y-%m-%d", localtime(time.time()))
        out_dir = self.output
        file_name =  self.config['model.name']  + '@' + current_time + '-weight' + f"-{mode}"+ '.pth'
        weight_file = out_dir + '/' + file_name 
        torch.save(model.state_dict(), weight_file)
        
    def save_performance_row(self, ep, data_ep):
        # opening the csv file in 'w' mode
        csv_path = self.output + 'train_performance.csv'
        
        # 'Hit Ratio:0.00328', 'Precision:0.00202', 'Recall:0.00337', 'NDCG:0.00292
        hit = float(data_ep[0].split(':')[1])
        precision = float(data_ep[1].split(':')[1])
        recall = float(data_ep[2].split(':')[1])
        ndcg = float(data_ep[3].split(':')[1])
        
        with open(csv_path, 'a+', newline = '') as f:
            header = ['ep', 'hit@20', 'prec@20', 'recall@20', 'ndcg@20']
            writer = csv.DictWriter(f, fieldnames = header)
            # writer.writeheader()
            writer.writerow({
                 'ep' : ep,
                 'hit@20': hit,
                 'prec@20': precision,
                 'recall@20': recall,
                 'ndcg@20': ndcg,
            })
            
    def save_loss_row(self, data_ep):
        csv_path = self.output + 'loss.csv'
        with open(csv_path, 'a+', newline ='') as f:
            header = ['ep', 'train_loss', 'cf_loss', 'kg_loss']
            writer = csv.DictWriter(f, fieldnames = header)
            # writer.writeheader()
            writer.writerow({
                'ep' : data_ep[0],
                'train_loss': data_ep[1],
                 'cf_loss': data_ep[2],
                 'kg_loss': data_ep[3]
            })

    def save_loss(self, train_losses, cf_losses, kg_losses):
        df_train_loss = pd.DataFrame(train_losses, columns = ['ep', 'loss'])
        df_cf_loss = pd.DataFrame(cf_losses, columns = ['ep', 'loss'])
        df_kg_loss = pd.DataFrame(kg_losses, columns = ['ep', 'loss'])
        df_train_loss.to_csv(self.output + '/train_loss.csv')
        df_cf_loss.to_csv(self.output + '/cf_loss.csv')
        df_kg_loss.to_csv(self.output + '/kg_loss.csv')

    def save_perfomance_training(self, log_train):
        df_train_log = pd.DataFrame(log_train)
        df_train_log.to_csv(self.output + '/train_performance.csv')

## Knowledge

In [5]:
class Knowledge(Interaction):
    def __init__(self, conf, training, test, knowledge):
        super().__init__(conf, training, test)
        self.conf = conf 
        self.kg_data = knowledge

        self.entity = {}
        self.id2ent = {}

        self.userent = {}
        self.itement = {}
        
        self.u2id = {}
        self.id2u = {}
        
        self.i2id = {}
        self.id2i = {}
        
        self.relation = {}
        self.id2rel = {}

        self.cf_train_data = np.array(training)
        self.training_set_e = defaultdict(dict)

        self.construct_data()
        
        self.laplacian_type = 'random-walk'
        self.create_adjacency_dict()
        self.create_laplacian_dict()
        
        self.kg_interaction_mat = self.__create_sparse_knowledge_interaction_matrix()
        self.interaction_mat = self.__create_sparse_interaction_matrix()
        
        
    def construct_data(self):
        kg_data = self.kg_data
        n_relations = max(kg_data['r']) + 1
        inverse_kg_data = kg_data.copy()
        inverse_kg_data = inverse_kg_data.rename({'h': 't', 't': 'h'}, axis='columns')
        inverse_kg_data['r'] += n_relations

        kg_data = pd.concat([kg_data, inverse_kg_data], axis=0, ignore_index=True, sort=False)

        # remap user_id 
        kg_data['r'] += 2
        
        kg_train_data = pd.concat([kg_data, inverse_kg_data], axis=0, ignore_index=True, sort=False)
        self.n_entities = max(max(kg_train_data['h']), max(kg_train_data['t'])) + 1
        self.n_relations = max(kg_train_data['r']) + 1

        # add interactions to kg data
        cf2kg_train_data = pd.DataFrame(np.zeros((self.n_cf_train, 3), dtype=np.int32), columns=['h', 'r', 't'])
        cf2kg_train_data['h'] = self.cf_train_data[:,0]
        cf2kg_train_data['t'] = self.cf_train_data[:,1]

        inverse_cf2kg_train_data = pd.DataFrame(np.ones((self.n_cf_train, 3), dtype=np.int32), columns=['h', 'r', 't'])
        inverse_cf2kg_train_data['h'] = self.cf_train_data[:,1]
        inverse_cf2kg_train_data['t'] = self.cf_train_data[:,0]

        self.kg_train_data = pd.concat([kg_train_data, cf2kg_train_data, inverse_cf2kg_train_data], ignore_index=True)
        self.n_kg_train = len(self.kg_train_data)

        self.n_users_entities = int(max(max(self.kg_train_data['h']), max(self.kg_train_data['t'])) + 1)

        # construct kg dict
        h_list = []
        t_list = []
        r_list = []

        self.train_kg_dict = collections.defaultdict(list)
        self.train_relation_dict = collections.defaultdict(list)

        for idx, row in self.kg_train_data.iterrows():
            h, r, t = int(row['h']), int(row['r']), int(row['t'])
            h_list.append(h)
            t_list.append(t)
            r_list.append(r)

            if h not in self.entity:
                self.entity[h] = len(self.entity)
                self.id2ent[self.entity[h]] = h
                # check h co phai user hay item k
                if h in self.user:
                    self.userent[h] = len(self.userent)
                #     # self.id2userent[self.userent[h]] = h
                if h in self.item:
                    self.itement[h] = len(self.itement)
                #     # self.id2itement[self.itement[h]] = h

            if t not in self.entity:
                self.entity[t] = len(self.entity)
                self.id2ent[self.entity[t]] = t 
                # check h co phai user hay item k 
                if t in self.user:
                    self.userent[t] = len(self.userent)
                #     # self.id2userent[self.userent[t]] = t
                if t in self.item:
                    self.itement[t] = len(self.itement)
                #     # self.id2itement[self.itement[t]] = t
            if r not in self.relation:
                self.relation[r] = len(self.relation)
                self.id2rel[self.relation[r]] = r 
            
            self.training_set_e[t][h] = r
            self.train_kg_dict[h].append((t, r))
            self.train_relation_dict[r].append((h, t))
        
        self.h_list = torch.LongTensor(h_list).to(device)
        self.t_list = torch.LongTensor(t_list).to(device)
        self.r_list = torch.LongTensor(r_list).to(device)
        
        lst_user_entities = list(self.userent.keys())
        lst_item_entities = list(self.itement.keys())

        for idx, u in enumerate(lst_user_entities):
            self.u2id[u] = idx
            self.id2u[idx] = u
        for idx, i in enumerate(lst_item_entities):
            self.i2id[i] = idx
            self.id2i[idx] = i
        
    def get_entity_id(self, e):
        if e in self.entity:
            return self.entity[e]
    
    def __create_sparse_knowledge_interaction_matrix(self):
        """
            return a sparse adjacency matrix with the shape (entity number, entity number)
        """
        row, col, entries = [], [], []
        for idx, pair in self.kg_train_data.iterrows():
            head, tail = int(pair['h']), int(pair['t'])
            row += [head]
            col += [tail]
            entries += [1.0]
        interaction_mat = sp.csr_matrix((entries, (row, col)), shape=(self.n_users_entities, self.n_users_entities),dtype=np.float32)
        return interaction_mat
    
    def __create_sparse_interaction_matrix(self):
        row, col, entries = [], [], []
        for pair in self.training_data:
            head, tail  = int(pair[0]), int(pair[1])
            row += [head]
            col += [tail]
            entries += [1.0]
        interaction_mat = sp.csr_matrix((entries, (row, col)), shape=(self.n_users_entities, self.n_users_entities),dtype=np.float32) 
        return interaction_mat
    
    def convert_coo2tensor(self, coo):
        values = coo.data
        indices = np.vstack((coo.row, coo.col))

        i = torch.LongTensor(indices)
        v = torch.FloatTensor(values)
        shape = coo.shape
        return torch.sparse.FloatTensor(i, v, torch.Size(shape))
    
    def create_adjacency_dict(self):
        self.adjacency_dict = {}
        for r, ht_list in self.train_relation_dict.items():
            rows = [e[0] for e in ht_list]
            cols = [e[1] for e in ht_list]
            vals = [1] * len(rows)
            adj = sp.coo_matrix((vals, (rows, cols)), shape=(self.n_users_entities, self.n_users_entities))
            self.adjacency_dict[r] = adj
    
    def create_laplacian_dict(self):
        def symmetric_norm_lap(adj):
            rowsum = np.array(adj.sum(axis=1))

            d_inv_sqrt = np.power(rowsum, -0.5).flatten()
            d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0
            d_mat_inv_sqrt = sp.diags(d_inv_sqrt)

            norm_adj = d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt)
            return norm_adj.tocoo()

        def random_walk_norm_lap(adj):
            rowsum = np.array(adj.sum(axis=1))

            d_inv = np.power(rowsum, -1.0).flatten()
            d_inv[np.isinf(d_inv)] = 0
            d_mat_inv = sp.diags(d_inv)

            norm_adj = d_mat_inv.dot(adj)
            return norm_adj.tocoo()

        if self.laplacian_type == 'symmetric':
            norm_lap_func = symmetric_norm_lap
        elif self.laplacian_type == 'random-walk':
            norm_lap_func = random_walk_norm_lap
        
        self.laplacian_dict = {}
        for r, adj in self.adjacency_dict.items():
            self.laplacian_dict[r] = norm_lap_func(adj)

        # A_in = sum(self.laplacian_dict.values())
        # self.A_in = self.convert_coo2tensor(A_in.tocoo())

## Sampler

In [6]:
def next_batch_kg(data_kg, batch_size, n_negs=1):
    ptr = 0
    kg_data = data_kg.kg_train_data.to_numpy()
    kg_dict = data_kg.train_kg_dict

    exist_heads= kg_dict.keys()
    h_list = list(exist_heads)
    h_dict = {value: idx for idx, value in enumerate(h_list)}
    all_tails = list(set(kg_data[:,2]))
    data_size = len(kg_data)
    # Pre-compute positive tail sets and negative tails for each head
    pos_tail_sets = {head: set([it[0] for it in tails]) for head, tails in kg_dict.items()}
    # neg_tail_sets = {head: np.random.choice(list(all_tails - pos_tails), size=n_negs) for head, pos_tails in pos_tail_sets.items()}
    
    while ptr < data_size:
        if ptr + batch_size < data_size:
            batch_end = ptr + batch_size
        else:   
            batch_end = data_size
        
        heads, relations, tails = kg_data[ptr:batch_end, 0], kg_data[ptr:batch_end, 1], kg_data[ptr:batch_end, 2]
        
        ptr = batch_end
        h_idx, r_idx, pos_t_idx, neg_t_idx = [], [], [], []
        # time1 = datetime.datetime.now()
        h_idx = [h_dict[head] for head in heads]
        
        r_idx.extend([int(rel) for rel in relations])
        pos_t_idx.extend([int(tail) for tail in tails])
        for head in heads:
            neg_t = random.choice(all_tails)
            while neg_t in pos_tail_sets[head]:
                neg_t = random.choice(all_tails)
            neg_t_idx.append(h_dict[neg_t])

        h_idx  = torch.LongTensor(h_idx).to(device)
        r_idx  = torch.LongTensor(r_idx).to(device)
        pos_t_idx  = torch.LongTensor(pos_t_idx).to(device)
        neg_t_idx  = torch.LongTensor(neg_t_idx).to(device)
        yield h_idx, r_idx, pos_t_idx, neg_t_idx

In [7]:
def next_batch_unified(data, data_kg, batch_size, batch_size_kg, n_negs=1):
    ptr = 0

    cf_data = data.training_data
    cf_size = len(cf_data) 
    
    shuffle(training_data)
    
    kg_data = data_kg.kg_train_data.to_numpy()
    shuffle(kg_data)
    
    kg_dict = data_kg.train_kg_dict

    exist_heads= kg_dict.keys()
    h_list = list(exist_heads)
    h_dict = {value: idx for idx, value in enumerate(h_list)}
    all_tails = list(set(kg_data[:,2]))
    data_size = len(kg_data)
    # Pre-compute positive tail sets and negative tails for each head
    pos_tail_sets = {head: set([it[0] for it in tails]) for head, tails in kg_dict.items()}
    # neg_tail_sets = {head: np.random.choice(list(all_tails - pos_tails), size=n_negs) for head, pos_tails in pos_tail_sets.items()}
    
    while ptr < data_size:
        if ptr + batch_size_kg < data_size:
            batch_end = ptr + batch_size_kg
        else:   
            batch_end = data_size
        
        heads, relations, tails = kg_data[ptr:batch_end, 0], kg_data[ptr:batch_end, 1], kg_data[ptr:batch_end, 2]
        
        ptr = batch_end
        h_idx, r_idx, pos_t_idx, neg_t_idx = [], [], [], []
        # time1 = datetime.datetime.now()
        h_idx = [h_dict[head] for head in heads]
        
        r_idx.extend([int(rel) for rel in relations])
        pos_t_idx.extend([int(tail) for tail in tails])
        for head in heads:
            neg_t = random.choice(all_tails)
            while neg_t in pos_tail_sets[head]:
                neg_t = random.choice(all_tails)
            neg_t_idx.append(h_dict[neg_t])

        # select random items
        selected_indices = np.random.choice(cf_size, batch_size)
        users = [training_data[idx][0] for idx in selected_indices]
        items = [training_data[idx][1] for idx in selected_indices]
        
        u_idx, i_idx, j_idx = [], [], []
        item_list = list(data.item.keys())
        for i, user in enumerate(users):
            i_idx.append(data.item[items[i]])
            u_idx.append(data.user[user])
            for m in range(n_negs):
                neg_item = choice(item_list)
                while neg_item in data.training_set_u[user]:
                    neg_item = choice(item_list)
                j_idx.append(data.item[neg_item])

        u_idx  = torch.LongTensor(u_idx).to(device)
        i_idx  = torch.LongTensor(i_idx).to(device)
        j_idx  = torch.LongTensor(j_idx).to(device)

        h_idx  = torch.LongTensor(h_idx).to(device)
        r_idx  = torch.LongTensor(r_idx).to(device)
        neg_t_idx  = torch.LongTensor(neg_t_idx).to(device)
        yield u_idx, i_idx, j_idx, h_idx, r_idx, pos_t_idx, neg_t_idx, users, items
    

## Layers

In [8]:
class HGNNConv(nn.Module):
    def __init__(self, leaky, input_dim, hyper_dim, bias=False):
        super(HGNNConv, self).__init__()
        self.hyper_dim = hyper_dim
        self.act = nn.LeakyReLU(negative_slope=leaky).to(device)
        self.fc = nn.Linear(input_dim, hyper_dim ,bias=False).to(device) 
        
        self.ln1 = torch.nn.LayerNorm(hyper_dim).to(device)
        self.ln2 = torch.nn.LayerNorm(hyper_dim).to(device)
        
    def forward(self, adj, embeds):
        lat1 = self.ln1(self.fc(torch.spmm(adj.t(), embeds)))
        output = self.act((self.ln2(torch.spmm(adj, lat1))))
        return output

In [9]:
class Attention(torch.nn.Module):
    # This class module is a simple attention layer.
    def __init__(self, in_size, hidden_size=128):
        super(Attention, self).__init__()

        self.project = torch.nn.Sequential(
            torch.nn.Linear(in_size, hidden_size),
            torch.nn.Tanh(),
            torch.nn.Linear(hidden_size, hidden_size, bias=False),
        )

    def forward(self, z):
        w = self.project(z)  # (N, 2, D)
        beta = torch.softmax(w, dim=1)  # (N, 2, D)
        return (beta * z).sum(1), beta  # (N, D), (N, 2, D)

## Model

In [10]:
class Model(nn.Module):
    def __init__(self, config, data, data_kg, args):
        super(Model, self).__init__()
        self.data = data
        adj = data.interaction_mat
        self.data_kg = data_kg 
        kg_adj = data_kg.kg_interaction_mat
        self.adj  = TorchGraphInterface.convert_sparse_mat_to_tensor(adj).to(device)
        self.kg_adj = TorchGraphInterface.convert_sparse_mat_to_tensor(kg_adj).to(device)

        self.user_indices =  torch.LongTensor(list(data.user.keys())).to(device)
        self.item_indices = torch.LongTensor(list(data.item.keys())).to(device)
        
        self._parse_args(args)
        self.embedding_dict = self._init_model()
        
        self.fc_u = nn.Linear(self.input_dim, self.hyper_dim)
        self.fc_i = nn.Linear(self.input_dim, self.hyper_dim)
        self.fc_e = nn.Linear(self.input_dim, self.hyper_dim)
        
        self.hgnn_u = [HGNNConv(leaky=self.p, input_dim=self.hyper_dim, hyper_dim=self.hyper_dim) for i in range(self.layers)]
        self.hgnn_i = [HGNNConv(leaky=self.p, input_dim=self.hyper_dim, hyper_dim=self.hyper_dim) for i in range(self.layers) ] 
        self.hgnn_e = [HGNNConv(leaky=self.p, input_dim=self.hyper_dim, hyper_dim=self.hyper_dim) for i in range(self.layers) ] 
        
        self.relation_emb =   nn.Parameter(init.xavier_uniform_(torch.empty(self.data_kg.n_relations, self.input_dim))).to(device)
        self.trans_M = nn.Parameter(init.xavier_uniform_(torch.empty(self.data_kg.n_relations, self.hyper_dim, self.relation_dim))).to(device)

        self.non_linear = nn.ReLU()
        self.act = nn.LeakyReLU(self.p)
        self.dropout = nn.Dropout(self.drop_rate)
        self.apply(self._init_weights)
        
    def _parse_args(self, args):
        self.input_dim = args['input_dim']
        self.hyper_dim = args['hyper_dim']
        self.p = args['p']
        self.drop_rate = args['drop_rate'] 
        self.layers = args['n_layers']
        self.temp = args['temp']
        self.aug_type = args['aug_type']
        self.relation_dim = args['relation_dim']
        
    def _init_model(self):
        initializer = init.xavier_uniform_
        embedding_dict = nn.ParameterDict({
            'user_entity_emb': nn.Parameter(initializer(torch.empty(self.data_kg.n_users_entities, self.input_dim)).to(device)),
            'entity_emb': nn.Parameter(initializer(torch.empty(self.data_kg.n_relations, self.input_dim)).to(device))
        })
        return embedding_dict
    
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            init.normal_(m.weight, std=0.01)
            if m.bias is not None:
                init.zeros_(m.bias)

    def graph_reconstruction(self):
        if self.aug_type==0 or 1:
            dropped_adj = self.random_graph_augment()
        else:
            dropped_adj = [], []
            for k in range(self.n_layers):
                dropped_adj = self.random_graph_augment()
                dropped_adj.append(dropped_adj_)
        return dropped_adj

    def random_graph_augment(self):
        dropped_mat = None
        if self.aug_type == 0:
            dropped_mat = GraphAugmentor.node_dropout(self.data.interaction_mat, self.drop_rate)
        elif self.aug_type == 1 or self.aug_type == 2:
            dropped_mat = GraphAugmentor.edge_dropout(self.data.interaction_mat, self.drop_rate)
        return TorchGraphInterface.convert_sparse_mat_to_tensor(dropped_mat).to(device)

    def calculate_cf_embeddings(self, perturbed_adj=None):
        uEmbed = self.embedding_dict['user_entity_emb'][self.user_indices]
        iEmbed = self.embedding_dict['user_entity_emb'][self.item_indices]
        
        uEmbed = self.dropout(self.act(self.fc_u(uEmbed)))
        iEmbed = self.dropout(self.act(self.fc_i(iEmbed)))
        
        embeds = torch.cat([uEmbed, iEmbed], 0)
        all_embeddings = [embeds]
        
        for k in range(self.layers):     
            if perturbed_adj is not None:
                if isinstance(perturbed_adj, list):
                    hyperULat = self.hgnn_u[k](perturbed_adj[k], uEmbed)
                    hyperILat = self.hgnn_i[k](perturbed_adj[k].t(), iEmbed)
                else:
                    hyperULat = self.hgnn_u[k](perturbed_adj, uEmbed)
                    hyperILat = self.hgnn_i[k](perturbed_adj.t(), iEmbed)
            else:
                hyperULat = self.hgnn_u[k](self.adj, uEmbed)
                hyperILat = self.hgnn_i[k](self.adj.t(), iEmbed)
            
            uEmbed += hyperULat
            iEmbed += hyperILat
            ego_embeddings = torch.cat([hyperULat, hyperILat], dim=0)
            all_embeddings += [ego_embeddings]
            
        all_embeddings = torch.stack(all_embeddings, dim=1)
        all_embeddings = torch.mean(all_embeddings, dim=1)
        user_all_embeddings = all_embeddings[:self.data.n_users]
        item_all_embeddings = all_embeddings[self.data.n_users:]
        return user_all_embeddings, item_all_embeddings 

    def calculate_kg_embeddings(self, perturbed_adj=None):
        eEmbed = self.embedding_dict['user_entity_emb']
        eEmbed = self.dropout(self.act(self.fc_e(eEmbed)))
        all_embeddings = [eEmbed]
        for k in range(self.layers):     
            if perturbed_adj is not None:
                if isinstance(perturbed_adj, list):
                    hyperELat = self.hgnn_e[k](perturbed_adj[k], eEmbed)
                else:
                    hyperELat = self.hgnn_e[k](perturbed_adj, eEmbed)
            else:
                hyperELat = self.hgnn_e[k](self.kg_adj, eEmbed)
            eEmbed += hyperELat
            all_embeddings += [eEmbed]
        all_embeddings = torch.stack(all_embeddings, dim=1)
        all_embeddings = torch.mean(all_embeddings, dim=1)
        return all_embeddings 

    def update_attention_batch(self, ego_embed, h_list, t_list, r_idx):
        r_embed = self.relation_emb[r_idx]
        W_r = self.trans_M[r_idx]
        h_embed = ego_embed[h_list]
        t_embed = ego_embed[t_list]
        # Equation (4)
        r_mul_h = torch.matmul(h_embed, W_r)
        r_mul_t = torch.matmul(t_embed, W_r)
        v_list = torch.sum(r_mul_t * torch.tanh(r_mul_h + r_embed), dim=1)
        return v_list
    
    def update_attention(self, ego_embed, h_list, t_list, r_list, relations):
        rows, cols, values = [], [], []

        for r_idx in relations:
            index_list = torch.where(r_list == r_idx)
            batch_h_list = h_list[index_list]
            batch_t_list = t_list[index_list]

            batch_v_list = self.update_attention_batch(ego_embed, batch_h_list, batch_t_list, r_idx)
            rows.append(batch_h_list)
            cols.append(batch_t_list)
            values.append(batch_v_list)

        rows = torch.cat(rows)
        cols = torch.cat(cols)
        values = torch.cat(values)

        indices = torch.stack([rows, cols])
        shape = self.kg_adj.shape
        A_in = torch.sparse.FloatTensor(indices, values, torch.Size(shape))
        # Equation (5)
        A_in = torch.sparse.softmax(A_in.cpu(), dim=1)
        self.kg_adj.data = A_in.to(device)

    def forward(self, perturbed_adj=None, mode='cf'):
        if mode == 'cf':
            user_embed, item_embed = self.calculate_cf_embeddings(perturbed_adj)
            return user_embed, item_embed
        elif mode == 'kg':
            entity_embed = self.calculate_kg_embeddings(perturbed_adj)
            return entity_embed 

    def calculate_cf_loss(self, anchor_emb, pos_emb, neg_emb, reg):
        calc_reg_loss = EmbLoss()
        rec_loss = bpr_loss(anchor_emb, pos_emb, neg_emb)
        reg_loss = reg * calc_reg_loss(anchor_emb, pos_emb, neg_emb)

        cf_loss  = rec_loss + reg_loss
        return cf_loss
    
    def calculate_kg_loss(self, h_embed, r, pos_t_embed, neg_t_embed, reg_kg):
        calc_reg_loss = EmbLoss()
        
        r_embed = self.relation_emb[r]                                                # (kg_batch_size, relation_dim)
        W_r = self.trans_M[r]                                                           # (kg_batch_size, embed_dim, relation_dim)
        r_mul_h = torch.bmm(h_embed.unsqueeze(1), W_r).squeeze(1)                       # (kg_batch_size, relation_dim)
        r_mul_pos_t = torch.bmm(pos_t_embed.unsqueeze(1), W_r).squeeze(1)               # (kg_batch_size, relation_dim)
        r_mul_neg_t = torch.bmm(neg_t_embed.unsqueeze(1), W_r).squeeze(1)               # (kg_batch_size, relation_dim)

        # Equation (1)
        pos_score = torch.sum(torch.pow(r_mul_h + r_embed - r_mul_pos_t, 2), dim=1)     # (kg_batch_size)
        neg_score = torch.sum(torch.pow(r_mul_h + r_embed - r_mul_neg_t, 2), dim=1)     # (kg_batch_size)

        # Equation (2)
        # kg_loss = F.softplus(pos_score - neg_score)
        kg_loss = (-1.0) * F.logsigmoid(neg_score - pos_score)
        kg_loss = torch.mean(kg_loss)

        reg_loss =  reg_kg * calc_reg_loss(r_mul_h, r_embed, r_mul_pos_t, r_mul_neg_t)
        loss = kg_loss + reg_loss
        return loss
        
    def cal_cl_loss(self, idxs, perturbed_mat1, perturbed_mat2):
        if type(idxs[0]) is not list:
            u_idx = torch.unique(idxs[0])
        else:
            u_idx = torch.unique(torch.Tensor(idxs[0]).to(device).type(torch.long))
        if type(idxs[1]) is not list:
            i_idx = torch.unique(idxs[1])
        else:
            i_idx = torch.unique(torch.Tensor(idxs[1]).to(device).type(torch.long))
        user_view_1, item_view_1 = self.forward(perturbed_mat1)
        user_view_2, item_view_2 = self.forward(perturbed_mat2)
        view1 = torch.cat((user_view_1[u_idx],item_view_1[i_idx]),0)
        view2 = torch.cat((user_view_2[u_idx],item_view_2[i_idx]),0)
        return InfoNCE(view1,view2,self.temp)

    

## Util

In [11]:
def calculate_ssl_loss(data, ancs, poss, emb_cf, emb_kg, temp):
    embeds1 = emb_cf
    embeds2 = emb_kg
    sslLoss = contrastLoss(embeds1[:data.n_users], embeds2[:data.n_users], torch.unique(ancs), temp) + \
                contrastLoss(embeds2[data.n_users:], embeds2[data.n_users:], torch.unique(poss), temp)
    return sslLoss

In [12]:
def predict(u, rec, user_emb, item_emb):
    user_id  = rec.data.get_user_id(u)
    score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
    return score.cpu().numpy()  

## Train

In [13]:
def train(train_model_cf, train_model_kg, attention_user, attention_item, rec, args):
    lst_train_losses = []
    lst_cf_losses = []
    lst_kg_losses = []
    lst_cl_losses = [] 
    
    lst_performances = []
    recall_list = []
    
    for ep in range(maxEpoch):        
        cf_losses = []
        kg_losses = []
        cl_losses = [] 
        
        cf_total_loss = 0
        kg_total_loss = 0
        cl_total_loss = 0 

        n_cf_batch = int(rec.data.n_cf_train // batchSize + 1)
        n_kg_batch = int(rec.data_kg.n_kg_train // batchSizeKG + 1)

        train_model_cf.train()
        train_model_kg.train()
        
        for n, batch in enumerate(next_batch_unified(rec.data, rec.data_kg, batchSize, batchSizeKG)):
            user_idx, pos_idx, neg_idx, kg_batch_head, kg_batch_relation, kg_batch_pos_tail, kg_batch_neg_tail, userent_id, itement_id = batch
            user_emb_cf, item_emb_cf = train_model_cf(mode='cf')

            ego_embed = train_model_kg(mode='kg')
            user_emb_kg, item_emb_kg = ego_embed[train_model_kg.user_indices], ego_embed[train_model_kg.item_indices]
            
            # anchor_emb_kg = ego_embed[userent_id]
            # pos_emb_kg = ego_embed[itement_id]
            
            kg_batch_head_emb = ego_embed[kg_batch_head]
            kg_batch_pos_tail_emb = ego_embed[kg_batch_pos_tail]
            kg_batch_neg_tail_emb = ego_embed[kg_batch_neg_tail]

            user_emb_fused, _ = attention_user(torch.stack([user_emb_cf, user_emb_kg], dim=1))
            item_emb_fused, _ = attention_item(torch.stack([item_emb_cf, item_emb_kg], dim=1))

            h_cf = torch.cat([user_emb_cf, item_emb_cf], dim=0)
            h_kg = torch.cat([user_emb_kg, item_emb_kg], dim=0)
            
            anchor_emb = user_emb_fused[user_idx]
            pos_emb = item_emb_fused[pos_idx]
            neg_emb = item_emb_fused[neg_idx]

            cf_batch_loss = train_model_cf.calculate_cf_loss(anchor_emb, pos_emb, neg_emb, reg)
            kg_batch_loss = train_model_kg.calculate_kg_loss(kg_batch_head_emb, kg_batch_relation, kg_batch_pos_tail_emb, kg_batch_neg_tail_emb, reg_kg)
            cf_total_loss += cf_batch_loss.item()
            kg_total_loss +=  kg_batch_loss.item()
            
            if args['use_contrastive']:
                cl_batch_loss = cl_rate * calculate_ssl_loss(rec.data, user_idx, pos_idx, h_cf, h_kg, args['temp'])
                cl_losses.append(cl_batch_loss.item())
                cl_total_loss += cl_batch_loss.item()
                batch_loss = cf_batch_loss + kg_batch_loss + cl_batch_loss
            else:
                batch_loss = cf_batch_loss + kg_batch_loss 

            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

            cf_losses.append(cf_batch_loss.item())
            kg_losses.append(kg_batch_loss.item())

            if (n % 20) == 0:
                print('CF Training: Epoch {:04d} Iter {:04d} / {:04d} | Iter Loss {:.4f} | Iter Mean Loss {:.4f}'.format(ep, n, n_kg_batch,  cf_batch_loss.item(), cf_total_loss / (n+1)))
                print('KG Training: Epoch {:04d} Iter {:04d} / {:04d} | Iter Loss {:.4f} | Iter Mean Loss {:.4f}'.format(ep, n, n_kg_batch, kg_batch_loss.item(), kg_total_loss / (n+1)))
                print('CL Training: Epoch {:04d} Iter {:04d} / {:04d} | Iter Loss {:.4f} | Iter Mean Loss {:.4f}'.format(ep, n, n_kg_batch, cl_batch_loss.item(), cl_total_loss / (n+1)))
                    
            if args['use_attention_propagation']:
                h_list  = rec.data_kg.h_list
                t_list  = rec.data_kg.t_list
                r_list = rec.data_kg.r_list
                relations = list(rec.data_kg.laplacian_dict.keys())
                train_model_kg.update_attention(ego_embed, h_list, t_list, r_list, relations)
        
        cf_loss = np.mean(cf_losses)

        kg_loss = np.mean(kg_losses)

        if args['use_contrastive']:
            cl_loss = np.mean(cl_losses)
            train_loss = cf_loss + kg_loss + cl_loss 
        else:
            cl_loss  = 0
            train_loss = cf_loss + kg_loss

        lst_cf_losses.append([ep,cf_loss])
        lst_kg_losses.append([ep, kg_loss])
        lst_train_losses.append([ep, train_loss])
        scheduler.step(train_loss)
        
        # Evaluation
        train_model_cf.eval()
        train_model_kg.eval()
        attention_user.eval()
        attention_item.eval()
        
        with torch.no_grad():
            user_emb_cf, item_emb_cf = train_model_cf(mode='cf')
            ego_emb = train_model_kg(mode='kg')
            user_emb_kg, item_emb_kg = ego_emb[train_model_kg.user_indices], ego_emb[train_model_kg.item_indices]

            user_emb, _ = attention_user(torch.stack([user_emb_cf, user_emb_kg], dim=1))
            item_emb, _ = attention_item(torch.stack([item_emb_cf, item_emb_kg], dim=1))
            data_ep = rec.fast_evaluation(ep, train_model_cf, train_model_kg, user_emb, item_emb)
        
            cur_recall =  float(data_ep[2].split(':')[1])
            recall_list.append(cur_recall)
            best_recall, should_stop = early_stopping(recall_list, 100)

        rec.save_performance_row(ep, data_ep)
        rec.save_loss_row([ep, train_loss, cf_loss, kg_loss, cl_loss])
        lst_performances.append(data_ep)

    rec.save_loss(lst_train_losses, lst_kg_losses, lst_cf_losses, lst_kg_losses)
    rec.save_perfomance_training(lst_performances)
    user_emb, item_emb = rec.best_user_emb, rec.best_item_emb
    return user_emb, item_emb

## Test

In [14]:
def test(rec, user_emb, item_emb):
    def process_bar(num, total):
        rate = float(num) / total
        ratenum = int(50 * rate)
        r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
        sys.stdout.write(r)
        sys.stdout.flush()

    # predict
    rec_list = {}
    user_count = len(rec.data.test_set)
    for i, user in enumerate(rec.data.test_set):
        # s_find_candidates = time.time()
        candidates = predict(user, rec, user_emb, item_emb)
        rated_list, li = rec.data.user_rated(user)
        for item in rated_list:
            candidates[rec.data.item[item]] = -10e8

        # s_find_k_largest = time.time()
        ids, scores = find_k_largest(rec.max_N, candidates)
        # e_find_k_largest = time.time()
        # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
        item_names = [rec.data.id2item[iid] for iid in ids]
        rec_list[user] = list(zip(item_names, scores))
        if i % 1000 == 0:
            process_bar(i, user_count)
    process_bar(user_count, user_count)
    print('')
    rec.evaluate(rec_list)

## Main

In [15]:
model = 'HGNN'
if model not in ['HGNN', 'LightGCN']:
    print("No model found.")
config = ModelConf('./conf/' + model + '.conf')

dataset = ['lastfm']
batchSize = 2048
batchSizeKG = 8192
maxEpoch = 1000
lRates = [0.01]
lRateKgs = [0.01]
lrDecays = [0.7]
regs = [0.01]
reg_kgs = [0.1]
hyperDims = [128]
inputDims = [32]
ps = [0.3]
dropRates = [0.3]
nLayers = [2]
nHeads = [1]
nSelfAtt = [1]
clRate = [0.01]
augType = [1]
temp = [0.3]
use_contrasts = [True]
use_attention_propagations = [True]
hyperparameters = [dataset, lRates, lRateKgs, lrDecays, regs, reg_kgs, hyperDims, inputDims, ps, dropRates, nLayers, nHeads, nSelfAtt, clRate, augType, temp, use_contrasts, use_attention_propagations]

In [16]:
dataset = 'lastfm'
training_data = FileIO.load_data_set('./dataset/' + dataset + '/' +config['training.set'], config['model.type'])
test_data = FileIO.load_data_set('./dataset/' + dataset + '/'  +config['test.set'], config['model.type'])
knowledge_set = FileIO.load_kg_data('./dataset/' + dataset +'/'+ dataset +'.kg')
data = Interaction(config, training_data, test_data)
data_kg = Knowledge(config, training_data, test_data, knowledge_set)

/tmp/ipykernel_18569/1351531476.py:181: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -1.0).flatten()
/tmp/ipykernel_18569/1351531476.py:181: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -1.0).flatten()
/tmp/ipykernel_18569/1351531476.py:181: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -1.0).flatten()
/tmp/ipykernel_18569/1351531476.py:181: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -1.0).flatten()
/tmp/ipykernel_18569/1351531476.py:181: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -1.0).flatten()
/tmp/ipykernel_18569/1351531476.py:181: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -1.0).flatten()
/tmp/ipykernel_18569/1351531476.py:181: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -1.0).flatten()
/tmp/ipykernel_18569/1351531476.py:181: RuntimeWarning:

In [ ]:
for params in product(*hyperparameters):
    dataset, lr, lr_kg, lr_decay, reg, reg_kg, hyper_dim, input_dim, prob, drop_rate, n_layers, n_heads, n_self_att, cl_rate, aug_type, temp, use_contrast, use_attention_propagation = params
    print(params)
    args = {
        'lr': lr,
        'lr_kg': lr_kg,
        'lr_decay': lr_decay,
        'reg': reg,
        'reg_kg': reg_kg,
        'hyper_dim': hyper_dim,
        'input_dim': input_dim,
        'p': prob,
        'drop_rate': drop_rate,
        'n_layers': n_layers,
        'input_dim': input_dim,
        'hyper_dim': hyper_dim,
        'relation_dim': 32,
        'n_heads': n_heads,
        'n_self_att': n_self_att,
        'dataset': dataset,
        'cl_rate': cl_rate,
        'aug_type': aug_type,
        'temp': temp,
        'use_contrastive': use_contrast,
        'use_attention_propagation': use_attention_propagation
    }
    rec = GraphRecommender(config, data, data_kg, knowledge_set, **args)
    train_model_cf = Model(rec.config, rec.data, rec.data_kg, args).to(device)
    train_model_kg = Model(rec.config, rec.data, rec.data_kg, args).to(device)
    attention_user = Attention(in_size=args['hyper_dim'], hidden_size=args['hyper_dim']).to(device)
    attention_item = Attention(in_size=args['hyper_dim'], hidden_size=args['hyper_dim']).to(device)

    para_list = (
        list(train_model_cf.parameters())
        + list(train_model_kg.parameters())
        + list(attention_user.parameters())
        + list(attention_item.parameters())
    )
    
    optimizer  = torch.optim.Adam(para_list, lr=lr)

    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=lr_decay, patience=10)

    user_emb, item_emb = train(train_model_cf, train_model_kg, attention_user,attention_item, rec, args)
    test(rec, user_emb, item_emb)
    

('lastfm', 0.01, 0.01, 0.7, 0.01, 0.1, 128, 32, 0.3, 0.3, 2, 1, 1, 0.01, 1, 0.3, True, True)


/mnt/disk1/hungvv/recommender/SELFRec/base/torch_interface.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  i = torch.LongTensor([coo.row, coo.col])


CF Training: Epoch 0000 Iter 0000 / 0263 | Iter Loss 0.6250 | Iter Mean Loss 0.6250
KG Training: Epoch 0000 Iter 0000 / 0263 | Iter Loss 0.7184 | Iter Mean Loss 0.7184
CL Training: Epoch 0000 Iter 0000 / 0263 | Iter Loss 0.1418 | Iter Mean Loss 0.1418
CF Training: Epoch 0000 Iter 0020 / 0263 | Iter Loss 0.3082 | Iter Mean Loss 0.4691
KG Training: Epoch 0000 Iter 0020 / 0263 | Iter Loss 0.5231 | Iter Mean Loss 0.5756
CL Training: Epoch 0000 Iter 0020 / 0263 | Iter Loss 0.1423 | Iter Mean Loss 0.1424
CF Training: Epoch 0000 Iter 0040 / 0263 | Iter Loss 0.2064 | Iter Mean Loss 0.3580
KG Training: Epoch 0000 Iter 0040 / 0263 | Iter Loss 0.4601 | Iter Mean Loss 0.5323
CL Training: Epoch 0000 Iter 0040 / 0263 | Iter Loss 0.1407 | Iter Mean Loss 0.1419
CF Training: Epoch 0000 Iter 0060 / 0263 | Iter Loss 0.1390 | Iter Mean Loss 0.2960
KG Training: Epoch 0000 Iter 0060 / 0263 | Iter Loss 0.4193 | Iter Mean Loss 0.5031
CL Training: Epoch 0000 Iter 0060 / 0263 | Iter Loss 0.1402 | Iter Mean Loss

In [ ]:
# del train_model_cf
# del train_model_kg
torch.cuda.empty_cache() # PyTorch thing
